# Miniproject 1
Duke University: ECE 580
Francisco Reveriano

In [1]:
# Import Necessary Libraries
import numpy as np
import matplotlib.pyplot as plt
from DCT.DCT_Matrix import *
from Regression_compressive_sensing import imgRead, imgShow, imgRecover
from sklearn.feature_extraction import image
from DCT.DCT_Matrix import DCT_Matrix
from util.util import *
from MOSEK.mosek import *

Original Image
X: 200 Y: 192
Transformation matrix: (16, 16)
Patch Original Shape: (4, 4)
First C Flatten Shape: (16,)
Mask Shape: (16,)
Mask Non-Sparse Values: 7
C Non-Sparse Values: 7
B Non-Sparse Values: 7
A Non-Sparse Values: 7
B Shape: (7,)
A Shape: (7, 16)
Alpha Shape: (16,)
[[176 177 178 179]
 [177 176 177 178]
 [176 176 176 178]
 [177 177 178 177]]
[[180.41421356 177.         178.         179.        ]
 [177.         176.         178.         178.        ]
 [172.17157288 174.58578644 178.         176.58578644]
 [168.75735931 173.58578644 178.         175.58578644]]


## Load Images

We First Proceed To Load the Boat Image. We then check the dimensions to make sure that the dimensions are single channel. We can then proceed to process the image. 

In [2]:
# Read/Load Images
boat = 'fishing_boat.bmp'
lena = "lena.bmp"

## Read the Boat Image
matrix = imgRead(boat)
P, Q = matrix.shape
print("Boat Image Dimensions")
print("X:", P, "Y",Q)

Boat Image Dimensions
X: 200 Y 192


## Test DCT Matrix

I am going to first test the performance of the DCT Matrix by Testing a single patch of the Image. 

### Read The Boat Image

In [3]:
## Read the Image Into Matrix
matrix = imgRead(boat)       
## Width and Length of Matrix
P, Q = matrix.shape                                                                                                     # Width and Length of Matrix
print("Original Image")
print("X:", P, "Y:",Q)

Original Image
X: 200 Y: 192


### Prepare DCT Matrix

In [4]:
# We First Declare the Dimensions for the T Matrix 
dimension = (4,4)
## We then call the DCT_Matrix function to create the DCT Matrix
T_Matrix = DCT_Matrix(dimension[0], dimension[1])
### We then Check that the T_Matrix is the correct dimensions
print("Transformation Shape")
print(T_Matrix.shape)

Transformation Shape
(16, 16)


### Split the Image Into Patches

I am utilizing Scikit learning Image Extract Patches 2D function. This is to make the process easier. 

In [5]:
patches_original_image = image.extract_patches_2d(matrix,dimension)                                                     # Turn into Patches the main Matrix
print("Patch Original Shape:",patches_original_image[0].shape)

Patch Original Shape: (4, 4)


### Declare C Matrix

In [6]:
# Flatten the First Patch to Create C
c = patches_original_image[0].flatten()
print("First C Flatten Shape:", c.shape)  

First C Flatten Shape: (16,)


### Make and Utilize the Mask

In [7]:
# Using Random Number Generator to Create Mask to Smaple
mask = np.random.randint(2, size=c.shape)
print("Mask Shape:",mask.shape)
print("Mask Non-Sparse Values:", count_non_sparse_values(mask))

# Get the New Sparse matrix
new_matrix = mask * c                                                                                                   # Taking Dot Product
## The Next Part is minimizing this matrix
C_values = count_non_sparse_values(new_matrix)                                                                          # Count the Values in the C Matrix that are non-sparse
print("C Non-Sparse Values:", C_values)
## Next part is creating the new sparse C matrix
B_Matrix = convert_C_to_B(C_values, new_matrix)
B_values = count_non_sparse_values(B_Matrix)
print("B Non-Sparse Values:", B_values)
assert(B_values == B_Matrix.shape[0])

# Convert the T Matrix to A
A_Matrix = convert_T_to_A(mask, T_Matrix)
A_values = count_non_sparse_values(A_Matrix.T[0])
print("A Non-Sparse Values:", A_values)

# At this point we need to set up our optimizer
print("B Shape:", B_Matrix.shape)
print("A Shape:", A_Matrix.shape)


Mask Shape: (16,)
Mask Non-Sparse Values: 9
C Non-Sparse Values: 9
B Non-Sparse Values: 9
A Non-Sparse Values: 9
B Shape: (9,)
A Shape: (9, 16)


### Use Mosek
I utilize Mosek to solve the underdetermined linear system. 
We can see that our values are very similar. Utilizing the Discrete cosine technique. 

In [9]:
alpha, res = l1norm(A_Matrix,B_Matrix)
C = np.matmul(T_Matrix,alpha)
C = C.reshape((dimension))
C = np.around(C)

print(patches_original_image[0])
print(C)

[[176 177 178 179]
 [177 176 177 178]
 [176 176 176 178]
 [177 177 178 177]]
[[179. 177. 178. 177.]
 [177. 175. 177. 175.]
 [176. 176. 176. 177.]
 [177. 177. 178. 178.]]
